# ABS Quarterly Estimated Resident Population 3101

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-set-up" data-toc-modified-id="Python-set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python set-up</a></span></li><li><span><a href="#Get-data-from-ABS" data-toc-modified-id="Get-data-from-ABS-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get data from ABS</a></span></li><li><span><a href="#Plot" data-toc-modified-id="Plot-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Plot</a></span><ul class="toc-item"><li><span><a href="#National-and-State-Populations" data-toc-modified-id="National-and-State-Populations-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>National and State Populations</a></span></li></ul></li><li><span><a href="#Finished" data-toc-modified-id="Finished-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Finished</a></span></li></ul></div>

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import pandas as pd
import matplotlib.pyplot as plt

# local imports
from plotting import (
    finalise_plot,
    set_chart_dir,
    recalibrate_series,
    clear_chart_dir,
    plot_covid_recovery,
)
from abs_data_capture import (
    get_fs_constants,
    get_ABS_meta_and_data,
    find_id,
)

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

# plotting stuff
plt.style.use("fivethirtyeight")

In [2]:
catalogue_id = "3101"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)

## Get data from ABS

In [3]:
# do the data capture and extraction
erp_dict = get_ABS_meta_and_data(catalogue_id)
if erp_dict is None:
    sys.exit(-1)
erp_meta = erp_dict[META_DATA]

Found URL for a ZIP file on ABS web page
We need to download this file from the ABS ...
Saving ABS download to cache.
Extracting DataFrames from the zip-file ...


## Plot

In [4]:
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)

In [5]:
table_col = "Table"
did_col = "Data Item Description"
erp_phrase = "Estimated Resident Population ;  Persons ;"

### National and State Populations

In [6]:
table = "4"
erp_df = erp_dict[table]

states = (
    erp_meta.loc[
        (erp_meta[table_col] == table) & (erp_meta[did_col].str.contains(erp_phrase)),
        did_col,
    ]
    .str.replace(erp_phrase, "")
    .str.replace(" ;", "")
    .str.strip()
    .to_list()
)

for state in states:
    # get relevant data series
    selector = {
        table: table_col,
        erp_phrase: did_col,
        f";  {state} ;": did_col,  # Australia, South Australia, etc.
    }
    id, units = find_id(erp_meta, selector, verbose=False)
    series = erp_df[id]
    series.name = "Estimated Resident Population"
    units = "Number Persons" if units == "Persons" else units
    series, units = recalibrate_series(series, units)

    # plot complete series
    ax = series.plot(lw=2)
    title = f"Estimated Resident Population: {state}"
    finalise_plot(
        ax,
        title=title,
        ylabel=f"{units}",
        rfooter=f"{source} {table}",
        # show=True,
    )

    plot_covid_recovery(
        series,
        title=title,
        ylabel=f"{units}",
        tag="-recent-covid",
        rfooter=f"{source} {table}",
        # show=True,
    )

## Finished

In [7]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Fri Feb 24 2023 08:10:27

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.10.0

matplotlib: 3.7.0
pandas    : 1.5.3
sys       : 3.11.0 | packaged by conda-forge | (main, Jan 15 2023, 05:44:48) [Clang 14.0.6 ]

Watermark: 2.3.1



In [8]:
print("Finished")

Finished
